# Monte Carlo Variance Reduction Methods - Antithetic, Delta and Gamma-based Control Variates

<b> YouTube Tutorial </b> (Published: Feb 22, 2022): https://youtu.be/MNWlag--c1Y

In this tutorial we will investigate ways we can reduce the variance of results from a Monte Carlo simulation method when valuing financial derivatives. The mathematic notation and examples are from Les Clewlow and Chris Strickland's book Implementing Derivatives Models.

Unfortunately, although a great method for approximating option values with complex payoffs or high dimensionality, in order to get an acceptably accurate estimate we must perform a large number of simulations M. Instead we can lean on Variance Reduction methods which work on the same principles as that of hedging an option position. i.e. the variability of a hedged option portfolio will have a smaller variance that that of it's unhedged counterpart.

In [ ]:
## This is required for google colab
!pip install py_vollib

     |████████████████████████████████| 130 kB 6.1 MB/s 
  Created wheel for py-vollib: filename=py_vollib-1.0.1-py3-none-any.whl size=62855 sha256=80b61225528da9b0d423ee75c1a54ee205612d582d6db8ae5139fdd1b9c03ce0
  Stored in directory: /root/.cache/pip/wheels/2d/25/50/bc80b93c9a827ed9bef9d86f85365e1934bcbc0666b9f00c11
  Created wheel for py-lets-be-rational: filename=py_lets_be_rational-1.0.1-py3-none-any.whl size=24468 sha256=d510b71aceeb62d261ebd29f6762ceab73ae0d4f86012aede8fb2d8168fefc2b
  Stored in directory: /root/.cache/pip/wheels/1a/70/10/bf54a16badc528594c9510ef517badb765a29b651ea5652898
Successfully built py-vollib py-lets-be-rational


In [1]:
# Import dependencies
import time
import math
import datetime
import numpy as np
import pandas as pd
import scipy.stats as stats
from py_vollib.black_scholes import black_scholes as bs

/Users/emanueleelias/anaconda3/lib/python3.11/site-packages/py_lets_be_rational/numba_helper.py:10: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  return jit(*jit_args, **jit_kwargs)(fun)


## General Control Variate Equation

For J control variates we have:

$ \Large C_0\exp(rT) = C_T - \sum^J_{i=j}\beta_j cv_j + \eta$

where
- $\beta_j$ are factors to account for the "true" linear relationship between the option pay-off and the control variate $cv_j$
- $\eta$ accounts for errors:
    - discrete rebalancing
    - approximations in hedge sensitivities (calc. delta / gamma)
    
    
Option price as the sum of the linear relationships with J control variates
    
$ \large C_T =\beta_0 + \sum^J_{i=j}\beta_j cv_j + \eta$

where $\beta_0 = C_0\exp(rT)$ is the forward price of the option

If we perform M simulations at discrete time intervals N we can regard the pay-offs and control variates as samples of the linear relationship with some noise. We can estimate the true relationship between control variates and option pay-offs with least-squares regression:

$\beta = (X'X)^{-1}X'Y$

We don't want biased estimates of $\beta_j$ so these should be precomputed by least-squares regression to establish the relationship between types of control variates and options first. These estaimates of $\beta_j$ values can then be used for $cv_j$ for pricing any option.  

In [2]:
# initial derivative parameters
S = 101.15          #stock price
K = 98.01           #strike price
vol = 0.0991        #volatility (%)
r = 0.015            #risk-free rate (%)
N = 20              #number of time steps
M = 1000            #number of simulations

market_value = 3.86 #market price of option
T = ((datetime.date(2022,3,17)-datetime.date(2022,1,17)).days+1)/365    #time in years
print(T)

0.1643835616438356


## Sample Estimate - no variance reduction methods

In [3]:
start_time = time.time()

# Precompute constants
dt = T/N  # Passo temporale
nudt = (r - 0.5*vol**2)*dt  # Drift del logaritmo del prezzo
volsdt = vol*np.sqrt(dt)  # Parte stocastica del logaritmo del prezzo
lnS = np.log(S)  # Logaritmo del prezzo iniziale

# Monte Carlo Method
Z = np.random.normal(size=(N, M))  # Numeri casuali standard normali
delta_lnSt = nudt + volsdt*Z  # Incremento stocastico
lnSt = lnS + np.cumsum(delta_lnSt, axis=0)  # Calcolo cumulativo del logaritmo del prezzo
lnSt = np.concatenate((np.full(shape=(1, M), fill_value=lnS), lnSt))  # Aggiunta del valore iniziale

# Compute Expectation and SE
ST = np.exp(lnSt)  # Prezzo del sottostante simulato
CT = np.maximum(0, ST - K)  # Payoff per call
# PT = np.maximum(0, K - ST)  # Payoff per put

C0_se = np.exp(-r*T)*np.sum(CT[-1])/M  # Prezzo attualizzato per call
# P0_se = np.exp(-r*T)*np.sum(PT[-1])/M  # Prezzo attualizzato per put

sigma = np.sqrt(np.sum((np.exp(-r*T)*CT[-1] - C0_se)**2) / (M-1))  # Deviazione standard per call
# sigma = np.sqrt(np.sum((np.exp(-r*T)*PT[-1] - P0_se)**2) / (M-1))  # Deviazione standard per put

SE_se = sigma/np.sqrt(M)  # Errore standard

mc_time_se = time.time() - start_time  # Tempo di computazione

# Stampa del risultato
print("Sample Estimate: Call value is ${0} with SE +/- {1}".format(np.round(C0_se, 2), np.round(SE_se, 2)))  # Per call
# print("Sample Estimate: Put value is ${0} with SE +/- {1}".format(np.round(P0_se, 2), np.round(SE_se, 2)))  # Per put
print("Computation time is: ", round(mc_time_se, 4))


Sample Estimate: Call value is $4.03 with SE +/- 0.11
Computation time is:  0.0017


## Implementation of Antithetic Variate

To implement an antithetic variate we create a hypothetical asset which is perfectly negatively correlated with the original asset. Implementation is very simple, and if we consider the example of the European Call Option (as in last weeks video). Our simulated pay-offs are under the following $S_t$ dynamics:

$\large S_{t+\Delta t} = S_{t} \exp( \nu \Delta t + \sigma (z_{t+\Delta t}- z_t) )$

Where $(z_{t+\Delta t}- z_t) \sim N(0,\Delta t) \sim \sqrt{\Delta t} N(0,1) \sim \sqrt{\Delta t} \epsilon_i$

### Contract Simulation

- $\large C_{T,i} = max(0, S \exp( \nu \Delta T + \sigma \sqrt{T} (\epsilon_i) ) - K)$

- $\large \bar{C}_{T,i} = max(0, S \exp( \nu \Delta T + \sigma \sqrt{T} (-\epsilon_i) ) - K)$

In [4]:
start_time = time.time()

# Precompute constants
dt = T/N  # Passo temporale
nudt = (r - 0.5*vol**2)*dt  # Drift del logaritmo del prezzo
volsdt = vol*np.sqrt(dt)  # Parte stocastica del logaritmo del prezzo
lnS = np.log(S)  # Logaritmo del prezzo iniziale

# Monte Carlo Method
Z = np.random.normal(size=(N, M))  # Numeri casuali standard normali
delta_lnSt1 = nudt + volsdt*Z  # Incremento stocastico per la traiettoria normale
delta_lnSt2 = nudt - volsdt*Z  # Incremento stocastico per la traiettoria antitetica
lnSt1 = lnS + np.cumsum(delta_lnSt1, axis=0)  # Logaritmo dei prezzi simulati (normale)
lnSt2 = lnS + np.cumsum(delta_lnSt2, axis=0)  # Logaritmo dei prezzi simulati (antitetico)

# Compute Expectation and SE
ST1 = np.exp(lnSt1)  # Prezzi simulati (normale)
ST2 = np.exp(lnSt2)  # Prezzi simulati (antitetico)
CT = 0.5 * (np.maximum(0, ST1[-1] - K) + np.maximum(0, ST2[-1] - K))  # Payoff per call
# PT = 0.5 * (np.maximum(0, K - ST1[-1]) + np.maximum(0, K - ST2[-1]))  # Payoff per put

C0_av = np.exp(-r*T)*np.sum(CT)/M  # Prezzo attualizzato per call
# P0_av = np.exp(-r*T)*np.sum(PT)/M  # Prezzo attualizzato per put

sigma = np.sqrt(np.sum((np.exp(-r*T)*CT - C0_av)**2) / (M-1))  # Deviazione standard per call
# sigma = np.sqrt(np.sum((np.exp(-r*T)*PT - P0_av)**2) / (M-1))  # Deviazione standard per put

SE_av = sigma/np.sqrt(M)  # Errore standard

mc_time_av = time.time() - start_time  # Tempo di computazione

# Stampa del risultato
print("Call value is ${0} with SE +/- {1}".format(np.round(C0_av, 2), np.round(SE_av, 2)))  # Per call
# print("Put value is ${0} with SE +/- {1}".format(np.round(P0_av, 2), np.round(SE_av, 2)))  # Per put
print("Computation time is: ", round(mc_time_av, 4))


Call value is $3.83 with SE +/- 0.03
Computation time is:  0.0018


## Implementation of Delta-based Control Variates

$\large cv_1 = \sum^{N-1}_{i=0} \frac{\delta C_{t_i}}{\delta S}(S_{t_{i+1}} - {\mathbb E}[S_{t_i}])\exp{r(T-t_{i+1})}$

$\large C_{t_0}\exp{rT} = C_T + \beta_1 cv_1 + \eta$


where with GBM dynamics:
- ${\mathbb E}[S_{t_{i+1}}] = S_{t_{i}} \exp (r \Delta t_i)$
- $\beta_1 = -1$ which is the appropriate value where we have exact delta for European Option

In [5]:
def delta_calc(r, S, K, T, sigma, type="c"):
    "Calculate delta of an option"
    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    try:
        if type == "c":
            delta_calc = stats.norm.cdf(d1, 0, 1)
        elif type == "p":
            delta_calc = -stats.norm.cdf(-d1, 0, 1)
        return delta_calc
    except:
        print("Please confirm option type, either 'c' for Call or 'p' for Put!")

In [6]:
start_time = time.time()

# Precompute constants
dt = T/N  # Passo temporale
nudt = (r - 0.5*vol**2)*dt  # Drift del logaritmo del prezzo
volsdt = vol*np.sqrt(dt)  # Parte stocastica del logaritmo del prezzo

erdt = np.exp(r*dt)  # Fattore di attualizzazione
cv = 0  # Variabile di controllo inizializzata
beta1 = -1  # Peso per la variabile di controllo

# Monte Carlo Method
Z = np.random.normal(size=(N, M))  # Numeri casuali standard normali
delta_St = nudt + volsdt*Z  # Incremento stocastico
ST = S*np.cumprod(np.exp(delta_St), axis=0)  # Prezzo del sottostante simulato
ST = np.concatenate((np.full(shape=(1, M), fill_value=S), ST))  # Aggiunta del prezzo iniziale

# Calcolo del delta
deltaSt = delta_calc(r, ST[:-1].T, K, np.linspace(T, dt, N), vol, "c").T  # Delta per call
# deltaSt = delta_calc(r, ST[:-1].T, K, np.linspace(T, dt, N), vol, "p").T  # Delta per put

# Calcolo della variabile di controllo
cv = np.cumsum(deltaSt * (ST[1:] - ST[:-1] * erdt), axis=0)

# Calcolo del payoff
CT = np.maximum(0, ST[-1] - K) + beta1 * cv[-1]  # Payoff per call
# PT = np.maximum(0, K - ST[-1]) + beta1 * cv[-1]  # Payoff per put

# Prezzo attualizzato
C0_dv = np.exp(-r*T) * np.sum(CT) / M  # Prezzo attualizzato per call
# P0_dv = np.exp(-r*T) * np.sum(PT) / M  # Prezzo attualizzato per put

# Calcolo dell'errore standard
sigma = np.sqrt(np.sum((np.exp(-r*T)*CT - C0_dv)**2) / (M-1))  # Deviazione standard per call
# sigma = np.sqrt(np.sum((np.exp(-r*T)*PT - P0_dv)**2) / (M-1))  # Deviazione standard per put
SE_dv = sigma / np.sqrt(M)

mc_time_dv = time.time() - start_time  # Tempo di computazione

# Stampa del risultato
print("Call value is ${0} with SE +/- {1}".format(np.round(C0_dv, 2), np.round(SE_dv, 3)))  # Per call
# print("Put value is ${0} with SE +/- {1}".format(np.round(P0_dv, 2), np.round(SE_dv, 3)))  # Per put
print("Computation time is: ", round(mc_time_dv, 4))


Call value is $3.82 with SE +/- 0.007
Computation time is:  0.0029


## Gamma Based Control Variate

The control variate is a random variable whose expected value we know, which is correlated with the varaible we are trying to estimate.

In the same way as for $cv_1$ we can create other control variates, which are equivalent to other hedges.

For example a gamma-based control variate ($cv_2$):

$\large cv_2 = \sum^{N-1}_{i=0} \frac{\delta^2 C_{t_i}}{\delta S^2}((\Delta S_{t_{i+1}})^2 - {\mathbb E}[(\Delta S_{t_i})^2])\exp{r(T-t_{i+1})}$

Where ${\mathbb E}[(\Delta S_{t_i})^2] = S_{t_i}^2 (\exp([2r+\sigma^2]\Delta t_i)-2\exp(r\Delta t_i)+1)$

In [7]:
def gamma_calc(r, S, K, T, sigma):
    "Calculate delta of an option"
    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    try:
        gamma_calc = stats.norm.pdf(d1, 0, 1)/(S*sigma*np.sqrt(T))
        return gamma_calc
    except:
        print("Please confirm option type, either 'c' for Call or 'p' for Put!")

In [8]:
start_time = time.time()

# Precompute constants
dt = T/N  # Passo temporale
nudt = (r - 0.5*vol**2)*dt  # Drift del logaritmo del prezzo
volsdt = vol*np.sqrt(dt)  # Parte stocastica del logaritmo del prezzo
erdt = np.exp(r*dt)  # Fattore di attualizzazione
ergamma = np.exp((2*r + vol**2)*dt) - 2*erdt + 1  # Componente per gamma
beta2 = -0.5  # Peso per la seconda variabile di controllo

# Monte Carlo Method
Z = np.random.normal(size=(N, M))  # Numeri casuali standard normali
delta_St = nudt + volsdt*Z  # Incremento stocastico
ST = S*np.cumprod(np.exp(delta_St), axis=0)  # Prezzo simulato del sottostante
ST = np.concatenate((np.full(shape=(1, M), fill_value=S), ST))  # Aggiunta del prezzo iniziale

# Calcolo del gamma
gammaSt = gamma_calc(r, ST[:-1].T, K, np.linspace(T, dt, N), vol).T

# Calcolo della seconda variabile di controllo
cv2 = np.cumsum(gammaSt * ((ST[1:] - ST[:-1])**2 - ergamma * ST[:-1]**2), axis=0)

# Calcolo del payoff
CT = np.maximum(0, ST[-1] - K) + beta2 * cv2[-1]  # Payoff per call
# PT = np.maximum(0, K - ST[-1]) + beta2 * cv2[-1]  # Payoff per put

# Prezzo attualizzato
C0_gv = np.exp(-r*T) * np.sum(CT) / M  # Prezzo attualizzato per call
# P0_gv = np.exp(-r*T) * np.sum(PT) / M  # Prezzo attualizzato per put

# Calcolo dell'errore standard
sigma = np.sqrt(np.sum((np.exp(-r*T)*CT - C0_gv)**2) / (M-1))  # Deviazione standard per call
# sigma = np.sqrt(np.sum((np.exp(-r*T)*PT - P0_gv)**2) / (M-1))  # Deviazione standard per put
SE_gv = sigma / np.sqrt(M)

mc_time_gv = time.time() - start_time  # Tempo di computazione

# Stampa del risultato
print("Call value is ${0} with SE +/- {1}".format(np.round(C0_gv, 2), np.round(SE_gv, 3)))  # Per call
# print("Put value is ${0} with SE +/- {1}".format(np.round(P0_gv, 2), np.round(SE_gv, 3)))  # Per put
print("Computation time is: ", round(mc_time_gv, 4))


Call value is $3.96 with SE +/- 0.11
Computation time is:  0.0025


## Combined Antithetic and Delta Variates

$C_T = 0.5( max(0, S_{1,t} - K) + max(0, S_{2,t} - K) + \beta_1 cv_1)$

where $cv_1$ is delta variate but now we have to account for antithetic variates - two perfectly negatively correlated underlyings.

$cv_1 = 0.5 * \beta_1 * (cv_{11} + cv_{12})$

where:

 - $cv_{11} = \Delta_{S_{1,t}}[S_{1,t_{i+1}} - S_{1,t_{i}} \exp (r \Delta t_i)]$

 - $cv_{12} = \Delta_{S_{2,t}}[S_{2,t_{i+1}} - S_{2,t_{i}} \exp (r \Delta t_i)]$

In [1]:
start_time = time.time()

# Precompute constants
dt = T/N  # Passo temporale
nudt = (r - 0.5*vol**2)*dt  # Drift del logaritmo del prezzo
volsdt = vol*np.sqrt(dt)  # Parte stocastica del logaritmo del prezzo
erdt = np.exp(r*dt)  # Fattore di attualizzazione
beta1 = -1  # Peso per la variabile di controllo

# Monte Carlo Method
Z = np.random.normal(size=(N, M))  # Numeri casuali standard normali
delta_St1 = nudt + volsdt*Z  # Incremento stocastico per la prima traiettoria
delta_St2 = nudt - volsdt*Z  # Incremento stocastico per la seconda traiettoria antitetica
ST1 = S*np.cumprod(np.exp(delta_St1), axis=0)  # Prezzo simulato del sottostante (traiettoria 1)
ST2 = S*np.cumprod(np.exp(delta_St2), axis=0)  # Prezzo simulato del sottostante (traiettoria 2)
ST1 = np.concatenate((np.full(shape=(1, M), fill_value=S), ST1))  # Aggiunta del prezzo iniziale (traiettoria 1)
ST2 = np.concatenate((np.full(shape=(1, M), fill_value=S), ST2))  # Aggiunta del prezzo iniziale (traiettoria 2)

# Calcolo del delta per entrambe le traiettorie
deltaSt1 = delta_calc(r, ST1[:-1].T, K, np.linspace(T, dt, N), vol, "c").T  # Delta per call (traiettoria 1)
deltaSt2 = delta_calc(r, ST2[:-1].T, K, np.linspace(T, dt, N), vol, "c").T  # Delta per call (traiettoria 2)
# deltaSt1 = delta_calc(r, ST1[:-1].T, K, np.linspace(T, dt, N), vol, "p").T  # Delta per put (traiettoria 1)
# deltaSt2 = delta_calc(r, ST2[:-1].T, K, np.linspace(T, dt, N), vol, "p").T  # Delta per put (traiettoria 2)

# Calcolo delle variabili di controllo per entrambe le traiettorie
cv11 = np.cumsum(deltaSt1 * (ST1[1:] - ST1[:-1] * erdt), axis=0)  # Variabile di controllo (traiettoria 1)
cv12 = np.cumsum(deltaSt2 * (ST2[1:] - ST2[:-1] * erdt), axis=0)  # Variabile di controllo (traiettoria 2)

# Calcolo del payoff
CT = 0.5 * (np.maximum(0, ST1[-1] - K) + beta1 * cv11[-1]
            + np.maximum(0, ST2[-1] - K) + beta1 * cv12[-1])  # Payoff per call
# PT = 0.5 * (np.maximum(0, K - ST1[-1]) + beta1 * cv11[-1]
#             + np.maximum(0, K - ST2[-1]) + beta1 * cv12[-1])  # Payoff per put

# Prezzo attualizzato
C0_adv = np.exp(-r*T) * np.sum(CT) / M  # Prezzo attualizzato per call
# P0_adv = np.exp(-r*T) * np.sum(PT) / M  # Prezzo attualizzato per put

# Calcolo dell'errore standard
sigma = np.sqrt(np.sum((np.exp(-r*T)*CT - C0_adv)**2) / (M-1))  # Deviazione standard per call
# sigma = np.sqrt(np.sum((np.exp(-r*T)*PT - P0_adv)**2) / (M-1))  # Deviazione standard per put
SE_adv = sigma / np.sqrt(M)

mc_time_adv = time.time() - start_time  # Tempo di computazione

# Stampa del risultato
print("Call value is ${0} with SE +/- {1}".format(np.round(C0_adv, 2), np.round(SE_adv, 3)))  # Per call
# print("Put value is ${0} with SE +/- {1}".format(np.round(P0_adv, 2), np.round(SE_adv, 3)))  # Per put
print("Computation time is: ", round(mc_time_adv, 4))


NameError: name 'time' is not defined

## Combined Antithetic, Delta and Gamma Variates

$C_T = 0.5( max(0, S_{1,t} - K) + max(0, S_{2,t} - K) + \beta_1 cv_1 + \beta_2 cv_2)$

where $cv_1$ is delta variate and $cv_2$ is the gamma variate. When combined with antithetic technique you have to apply the following!

$cv_1 = 0.5 * \beta_1 * (cv_{11} + cv_{12})$

 - $cv_{11} = \Delta_{S_{1,t}}[S_{1,t_{i+1}} - S_{1,t_{i}} \exp (r \Delta t_i)]$

 - $cv_{12} = \Delta_{S_{2,t}}[S_{2,t_{i+1}} - S_{2,t_{i}} \exp (r \Delta t_i)]$

$cv_2 = 0.5 * \beta_2 * (cv_{21} + cv_{22})$

 - $cv_{21} = \gamma_{S_{1,t}}[(S_{1,t_{i+1}} - S_{1,t_i})^2 - S_{1,t_i}^2 (\exp([2r+\sigma^2]\Delta t_i)-2\exp(r\Delta t_i)+1)]$

 - $cv_{22} = \gamma_{S_{2,t}}[(S_{2,t_{i+1}} - S_{2,t_i})^2 - S_{2,t_i}^2 (\exp([2r+\sigma^2]\Delta t_i)-2\exp(r\Delta t_i)+1)]$

In [10]:
start_time = time.time()

# Precompute constants
dt = T/N  # Passo temporale
nudt = (r - 0.5*vol**2)*dt  # Drift del logaritmo del prezzo
volsdt = vol*np.sqrt(dt)  # Parte stocastica del logaritmo del prezzo
erdt = np.exp(r*dt)  # Fattore di attualizzazione
ergamma = np.exp((2*r + vol**2)*dt) - 2*erdt + 1  # Componente per gamma

beta1 = -1  # Peso per la variabile di controllo delta
beta2 = -0.5  # Peso per la variabile di controllo gamma

# Monte Carlo Method
Z = np.random.normal(size=(N, M))  # Numeri casuali standard normali
delta_St1 = nudt + volsdt*Z  # Incremento stocastico per la prima traiettoria
delta_St2 = nudt - volsdt*Z  # Incremento stocastico per la traiettoria antitetica
ST1 = S*np.cumprod(np.exp(delta_St1), axis=0)  # Prezzo simulato del sottostante (traiettoria 1)
ST2 = S*np.cumprod(np.exp(delta_St2), axis=0)  # Prezzo simulato del sottostante (traiettoria 2)
ST1 = np.concatenate((np.full(shape=(1, M), fill_value=S), ST1))  # Aggiunta del prezzo iniziale (traiettoria 1)
ST2 = np.concatenate((np.full(shape=(1, M), fill_value=S), ST2))  # Aggiunta del prezzo iniziale (traiettoria 2)

# Calcolo del delta per entrambe le traiettorie
deltaSt1 = delta_calc(r, ST1[:-1].T, K, np.linspace(T, dt, N), vol, "c").T  # Delta per call (traiettoria 1)
deltaSt2 = delta_calc(r, ST2[:-1].T, K, np.linspace(T, dt, N), vol, "c").T  # Delta per call (traiettoria 2)
# deltaSt1 = delta_calc(r, ST1[:-1].T, K, np.linspace(T, dt, N), vol, "p").T  # Delta per put (traiettoria 1)
# deltaSt2 = delta_calc(r, ST2[:-1].T, K, np.linspace(T, dt, N), vol, "p").T  # Delta per put (traiettoria 2)

# Calcolo del gamma per entrambe le traiettorie
gammaSt1 = gamma_calc(r, ST1[:-1].T, K, np.linspace(T, dt, N), vol).T  # Gamma (traiettoria 1)
gammaSt2 = gamma_calc(r, ST2[:-1].T, K, np.linspace(T, dt, N), vol).T  # Gamma (traiettoria 2)

# Calcolo delle variabili di controllo delta per entrambe le traiettorie
cv11 = np.cumsum(deltaSt1 * (ST1[1:] - ST1[:-1] * erdt), axis=0)  # Variabile di controllo delta (traiettoria 1)
cv12 = np.cumsum(deltaSt2 * (ST2[1:] - ST2[:-1] * erdt), axis=0)  # Variabile di controllo delta (traiettoria 2)

# Calcolo delle variabili di controllo gamma per entrambe le traiettorie
cv21 = np.cumsum(gammaSt1 * ((ST1[1:] - ST1[:-1])**2 - ergamma * ST1[:-1]**2), axis=0)  # Variabile di controllo gamma (traiettoria 1)
cv22 = np.cumsum(gammaSt2 * ((ST2[1:] - ST2[:-1])**2 - ergamma * ST2[:-1]**2), axis=0)  # Variabile di controllo gamma (traiettoria 2)

# Calcolo del payoff
CT = 0.5 * (np.maximum(0, ST1[-1] - K) + beta1 * cv11[-1] + beta2 * cv21[-1]
            + np.maximum(0, ST2[-1] - K) + beta1 * cv12[-1] + beta2 * cv22[-1])  # Payoff per call
# PT = 0.5 * (np.maximum(0, K - ST1[-1]) + beta1 * cv11[-1] + beta2 * cv21[-1]
#             + np.maximum(0, K - ST2[-1]) + beta1 * cv12[-1] + beta2 * cv22[-1])  # Payoff per put

# Prezzo attualizzato
C0_adgv = np.exp(-r*T) * np.sum(CT) / M  # Prezzo attualizzato per call
# P0_adgv = np.exp(-r*T) * np.sum(PT) / M  # Prezzo attualizzato per put

# Calcolo dell'errore standard
sigma = np.sqrt(np.sum((np.exp(-r*T)*CT - C0_adgv)**2) / (M-1))  # Deviazione standard per call
# sigma = np.sqrt(np.sum((np.exp(-r*T)*PT - P0_adgv)**2) / (M-1))  # Deviazione standard per put
SE_adgv = sigma / np.sqrt(M)

mc_time_adgv = time.time() - start_time  # Tempo di computazione

# Stampa del risultato
print("Call value is ${0} with SE +/- {1}".format(np.round(C0_adgv, 2), np.round(SE_adgv, 3)))  # Per call
# print("Put value is ${0} with SE +/- {1}".format(np.round(P0_adgv, 2), np.round(SE_adgv, 3)))  # Per put
print("Computation time is: ", round(mc_time_adgv, 4))


Call value is $3.82 with SE +/- 0.001
Computation time is:  0.0166


## Comparing Reduction Methods
### Reviewing Contract Details

In [11]:
params = [K,round(T,2),S,vol,r,N,M,round(bs('c', S, K, T, r, vol),2), market_value]
params_rd = [round(param,2) for param in params]

data = {'Contract Terms':['Strike Price', 'Time to Maturity', 'Asset Price', 'Volatility',
                          'Riskfree Rate', 'Number of Time Steps', 'Number of Simuations',
                          'Standard European Call Price', 'Market Price'],
        'Parameters': params_rd}

# Creates pandas DataFrame.
df = pd.DataFrame(data) #, index =['position1', 'position2', 'position3', 'position4'])

df

,Contract Terms,Parameters
0,Strike Price,98.01
1,Time to Maturity,0.16
2,Asset Price,101.15
3,Volatility,0.10
4,Riskfree Rate,0.01
5,Number of Time Steps,20.00
6,Number of Simuations,1000.00
7,Standard European Call Price,3.82
8,Market Price,3.86


### Trade-off between Error vs Computation Time

Comparison table of standard errors and relative computation time for each reduction method or combination

In [12]:
se_variates = [SE_se, SE_av, SE_dv, SE_gv, SE_adv, SE_adgv]
se_rd = [round(se,4) for se in se_variates]
se_red = [round(SE_se/se,2) for se in se_variates]

comp_time = [mc_time_se, mc_time_av, mc_time_dv, mc_time_gv, mc_time_adv, mc_time_adgv]
rel_time = [round(mc_time/mc_time_se,2) for mc_time in comp_time]

data = {'Standard Error (SE)': se_rd,
        'SE Reduction Multiple': se_red,
        'Relative Computation Time': rel_time}

# Creates pandas DataFrame.
df = pd.DataFrame(data, index =['Simple estimate', 'with antithetic variate',
'with delta-based control variate', 'with gamma-based control variate', 'with antithetic and delta variates', 'with all combined variates'])

df

,Standard Error (SE),SE Reduction Multiple,Relative Computation Time
Simple estimate,0.1096,1.00,1.00
with antithetic variate,0.0278,3.94,1.07
with delta-based control variate,0.0074,14.74,1.68
with gamma-based control variate,0.1097,1.00,1.46
with antithetic and delta variates,0.0059,18.58,2.03
with all combined variates,0.0014,76.33,9.73
